## 네이버 영화 페이지 크롤링
### 네이버 영화에 등록된 데이터 수집
- 제목, 평점, 장르, 감독, 배우, 개봉일, 상영시간 수집
- 각 요소 중 하나라도 없으면 크롤링 대상에서 제외
    - 영화 정보 제공의 필요성으로, 결측값 대체 없이 모든 요소가 존재할 때만 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
browser = webdriver.Chrome('./chromedriver.exe')

In [3]:
url = "https://movie.naver.com/movie/bi/mi/basic.nhn?code="
# uri = f"https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}" 

navermovies = []

# 크롤링하고자 하는 페이지 : 숫자 6자리
for page in range(184316,184567):
    try:       
        target = url+str(page)
#         print(target)
        browser.get(target)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.select("div.article")
        content = contents[0]
        
        # 영화 제목
        title = content.select_one("h3.h_movie>a")
        
        # 평점
        score = content.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
        non_score = "관람객 평점 없음"
        if (score == None) or (non_score in score.text):
            continue
        score = score.text
        score = score[6:11]

        # 장르
        genre = content.select("dl.info_spec dd p span:nth-of-type(1) a")

        # 감독
        directors = content.select("dl.info_spec dd:nth-of-type(2) a")

        # 배우 
        actors = content.select("dl.info_spec dd:nth-of-type(3) a")

        # 영화 개봉일
        rdate = content.select("dl.info_spec dd p span:nth-of-type(4):nth-child(n+3):nth-child(-n+4)")

        # 상영시간
        rtime = content.select_one("dl.info_spec dd p span:nth-of-type(3)")

        genre_list = [g.text for g in genre]
        directors_list = [d.text for d in directors]
        actors_list = [a.text for a in actors]
        rdate_list = [r.text for r in rdate]
        
        # 출연배우 다수일 때
        if '더보기' in actors_list :
            actors_list.remove('더보기')

        genre_str = ','.join(genre_list)
        directors_str = ','.join(directors_list)
        actors_str = ','.join(actors_list)
        rdate_str = ','.join(rdate_list)

        # 개봉일 전처리
        rdate_str = re.sub('[\s]', '', rdate_str)
        rdate_str = re.sub('개봉', '', rdate_str)
        if ',' in rdate_str :
            rdate_str = rdate_str.split(',')[1]
            
        # 최종 리스트 삽입
        navermovies.append([title.text.strip(), score.strip(), genre_str, directors_str, actors_str, rdate_str, rtime.text.strip()])
        
    except :
        print(page,"Page not found")
        #네이버에서 영화가 등록되지 않은 page number 확인 가능
        
len(navermovies)

184432 Page not found
184434 Page not found
184507 Page not found
184510 Page not found


8

In [6]:
# 리스트 형태 예제
navermovies[:5]

[['블랙 위도우',
  '9.09',
  '액션,모험,SF',
  '케이트 쇼트랜드',
  '스칼릿 조핸슨,플로렌스 퓨,레이첼 와이즈',
  '2021.07.07',
  '134분'],
 ['전주에서 길을 묻다', '10.0', '드라마', '진승현', '강양은,조성희,최수경', '2021.06.01', '91분'],
 ['마치다군의 세계',
  '9.00',
  '멜로/로맨스',
  '이시이 유야',
  '호소다 카나타,세키미즈 나기사',
  '2020.11.12',
  '120분'],
 ['아담스 패밀리',
  '8.44',
  '애니메이션,코미디,가족',
  '그렉 티어난,콘래드 버논',
  '샤를리즈 테론,클로이 모레츠,오스카 아이삭',
  '2019.11.07',
  '87분'],
 ['아무도 없는 곳', '8.14', '드라마', '김종관', '연우진,김상호,아이유', '2021.03.31', '83분']]

In [7]:
# 데이터프레임 형태로 저장
naver_data = pd.DataFrame(navermovies, columns = ['제목','관람객평점','장르', '감독', '출연배우', '개봉일','상영시간'])

In [8]:
# 저장되는 형식 예시
naver_data

,제목,관람객평점,장르,감독,출연배우,개봉일,상영시간
0,블랙 위도우,9.09,"액션,모험,SF",케이트 쇼트랜드,"스칼릿 조핸슨,플로렌스 퓨,레이첼 와이즈",2021.07.07,134분
1,전주에서 길을 묻다,10.0,드라마,진승현,"강양은,조성희,최수경",2021.06.01,91분
2,마치다군의 세계,9.00,멜로/로맨스,이시이 유야,"호소다 카나타,세키미즈 나기사",2020.11.12,120분
3,아담스 패밀리,8.44,"애니메이션,코미디,가족","그렉 티어난,콘래드 버논","샤를리즈 테론,클로이 모레츠,오스카 아이삭",2019.11.07,87분
4,아무도 없는 곳,8.14,드라마,김종관,"연우진,김상호,아이유",2021.03.31,83분
5,쓰리 세컨즈,9.80,드라마,안톤 메게르디체브,"블라디미르 마쉬코프,키릴 자이체프,이반 코레스니코프",2019.06.20,133분
6,소울,9.32,애니메이션,피트 닥터,"제이미 폭스,티나 페이,다비드 딕스",2021.01.20,107분
7,라야와 마지막 드래곤,9.44,"애니메이션,액션,모험","돈 홀,까를로스 로페즈 에스트라다","켈리 마리 트란,아콰피나,산드라 오",2021.03.04,114분


In [18]:
# 원하는 경로를 설정하여 저장
naver_data.to_excel('./data/naver.xls')

<ipython-input-18-a83d9815b8d0>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  naver_data.to_excel('./data/naver.xls')
